<a href="https://colab.research.google.com/github/lkarjun/malayalam-language-model/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dataset Loading

In [1]:
!pip install dvc[gdrive]

!dvc get https://github.com/lkarjun/malayalam-language-model \
datasets/

     |████████████████████████████████| 401 kB 5.1 MB/s 
     |████████████████████████████████| 49 kB 5.5 MB/s 
     |████████████████████████████████| 215 kB 48.8 MB/s 
     |████████████████████████████████| 109 kB 57.9 MB/s 
     |████████████████████████████████| 41 kB 36 kB/s 
     |████████████████████████████████| 133 kB 59.8 MB/s 
     |████████████████████████████████| 44 kB 2.4 MB/s 
     |████████████████████████████████| 48 kB 4.0 MB/s 
     |████████████████████████████████| 280 kB 56.8 MB/s 
     |████████████████████████████████| 4.5 MB 52.6 MB/s 
     |████████████████████████████████| 287 kB 67.6 MB/s 
     |████████████████████████████████| 180 kB 66.8 MB/s 
     |████████████████████████████████| 547 kB 75.7 MB/s 
     |████████████████████████████████| 1.1 MB 46.3 MB/s 
     |████████████████████████████████| 3.6 MB 48.7 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 55 kB 3.1 MB/s 
     |██████████████████████


    https://accounts.google.com/o/oauth2/auth?client_id=710796635688-iivsgbgsb6uv1fap6635dhvuei09o66c.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.appdata&access_type=offline&response_type=code&approval_prompt=force

Enter verification code: 4/1AX4XfWhjQsnXPsCL-FNLDMEBc6EWdLpehWqpkHYYLJ-0X3IEqUfBlSWLicA
Authentication successful.


In [2]:
!unzip 'datasets/*.zip' -d datasets/

Streaming output truncated to the last 5000 lines.
  inflating: datasets/magazine_raw/mag3577.txt  
  inflating: datasets/magazine_raw/mag3578.txt  
  inflating: datasets/magazine_raw/mag3579.txt  
  inflating: datasets/magazine_raw/mag358.txt  
  inflating: datasets/magazine_raw/mag3580.txt  
  inflating: datasets/magazine_raw/mag3581.txt  
  inflating: datasets/magazine_raw/mag3582.txt  
  inflating: datasets/magazine_raw/mag3583.txt  
  inflating: datasets/magazine_raw/mag3584.txt  
  inflating: datasets/magazine_raw/mag3585.txt  
  inflating: datasets/magazine_raw/mag3586.txt  
  inflating: datasets/magazine_raw/mag3587.txt  
  inflating: datasets/magazine_raw/mag3588.txt  
  inflating: datasets/magazine_raw/mag3589.txt  
  inflating: datasets/magazine_raw/mag359.txt  
  inflating: datasets/magazine_raw/mag3590.txt  
  inflating: datasets/magazine_raw/mag3591.txt  
  inflating: datasets/magazine_raw/mag3592.txt  
  inflating: datasets/magazine_raw/mag3593.txt  
  inflating: dataset

## Packages importing

In [13]:
import pandas as pd
import spacy
from tqdm import tqdm
from pathlib import Path
import textacy.preprocessing as tprep

tqdm.pandas()

# Dataset Loading

In [2]:
BASE_PATH = Path("datasets/")

mag = pd.read_csv(BASE_PATH/'magazine_files.csv')
wiki = pd.read_csv(BASE_PATH/'wikitext_files.csv')

In [3]:
mag.head(4)

,file_path,dataset_name,length
0,datasets/magazine_raw/mag0.txt,magazine_text,4383
1,datasets/magazine_raw/mag1.txt,magazine_text,1140
2,datasets/magazine_raw/mag10.txt,magazine_text,7582
3,datasets/magazine_raw/mag100.txt,magazine_text,2508


In [4]:
def open_files(path: str):
  with open(path, 'r') as file:
    txt = file.read()
  return txt

mag['content'] = mag['file_path'].progress_apply(open_files)
wiki['content'] = wiki['file_path'].progress_apply(open_files)

100%|██████████| 11341/11341 [00:02<00:00, 4935.45it/s]


In [5]:
mag.head(4)

,file_path,dataset_name,length,content
0,datasets/magazine_raw/mag0.txt,magazine_text,4383,"ഇന്ത്യയിലെ ആദ്യത്തെ വനിതാ ഐ‌എ‌എസ് ഉദ്യോഗസ്ഥയെ കുറിച്ചറിയാമോ? അവരുടെ പേരാണ് അന്ന രാജം മൽഹോത്ര. അവർ ഒരു മലയാളി കൂടിയാണ് എന്നത് നാം മലയാളികൾക്ക് അഭിമാനിക്കാനുള്ള വകയാണ്. കുഞ്ഞുകുട്ടി പ്രാരാബ്ദങ്ങളുമായി വീടുകളിൽ ഒതുങ്ങി കഴിഞ്ഞിരുന്ന സ്ത്രീകൾക്കിടയിൽ അന്ന രാജം മൽ‌ഹോത്ര വേറിട്ട് നിന്നു. സമൂഹത്തിൽ നിലനിന്നിരുന്ന അസമത്വത്തിനെതിരെ, ലിംഗ അനീതികൾക്കെതിരെ പോരാടിയ സ്ത്രീയാണ് അവർ, ഇന്ത്യയിലെ ആദ്യത്തെ വനിതാ ഐ‌എ‌എസ് ഉദ്യോഗസ്ഥ. കേന്ദ്രസർക്കാരിൽ സെക്രട്ടേറിയൽ പദവി വഹിച്ച ആദ്യ വനിത കൂടിയായിരുന്നു അവർ. വളരെ പ്രചോദനാത്മകമായ അവരുടെ കഥ. 1927 ജൂലൈ 17 -ന് കേരളത്തിലെ നിരണം ഗ്രാമത്തിൽ ജനിച്ച അന്ന, മദ്രാസ് സർവകലാശാലയ..."
1,datasets/magazine_raw/mag1.txt,magazine_text,1140,"ഒരു മ്യുസീഷന് ഡാന്‍സ് ചെയ്യാനാവും, ഡ്രം വായിക്കാനുമാവും. ഡ്രം വായിച്ചുകൊണ്ടിരിക്കെ ഡാന്‍സ് ചെയ്യാനാവും. ആ കലാകാരന്‍റെ വീഡിയോ ഇപ്പോള്‍ സാമൂഹിക മാധ്യമങ്ങളില്‍ വൈറലാവുകയാണ്. നിരവധിപ്പേരാണ് ഇദ്ദേഹത്തിന്‍റെ കഴിവുകളെ പ്രശംസിച്ചിട്ടു കൊണ്ട് മുന്നോട്ട് വന്നിരിക്കുന്നത്. തിമോത്തി ഫ്ലെച്ചര്‍ എന്നാണ് അദ്ദേഹത്തിന്‍റെ പേര്. 'ബാൾട്ടിമോറിന്‍റെ കുട്ടി, കൂടുതൽ എന്തെങ്കിലും ആവാന്‍ ആഗ്രഹിക്കുകയും അതിനായി പ്രവർത്തിക്കുകയും ചെയ്യുന്നയാളാണ്' എന്നാണ് തിമോത്തി ഫ്ലെച്ചറിന്‍റെ ഇന്‍സ്റ്റഗ്രാം ബയോ പറയുന്നത്. ഏതായാലും ഫ്ലെച്ചര്‍ പങ്കുവച്ച വീഡിയോ വളരെ ചുരുങ്ങിയ സമയം കൊണ്ടാണ് വൈറലായത്. A post shared by Formerly Known A..."
2,datasets/magazine_raw/mag10.txt,magazine_text,7582,"'മോളേ, ടോണിയെ അകത്ത് കയറ്റണോ?' 'ഹേയ്! ആള്‍ക്ക് ഇന്‍ഡ്യന്‍ പോണ്‍ ഇഷ്ടമല്ല. അവിടെ നിന്നോട്ടെ.' തലക്കടി കിട്ടിയത് പോലെ ടോണി കിട്ടിയ കസേരയിലിരുന്നു. പിന്നേയും എന്റെ വണ്ടിയുരുണ്ട്, തൂക്കിയിട്ടേക്കുന്ന ഒരു വലിയ ലൈറ്റിന് താഴെ കൊണ്ട് നിര്‍ത്തി. ഡോക്ടര്‍മാരുടേയും നഴ്‌സ്മാരുടേയും ഇടയില്‍ ഞാന്‍ കിടന്നു. 'ദൈവമേ, ഇവരുടെയൊക്കെ മുന്നില്‍ സില്‍ക് സ്മിതയുടെ ഉടുപ്പും ഇട്ടോണ്ടാണല്ലോ ഞാന്‍ കെടക്കണത്!' ഞാന്‍ വീണ്ടും കണ്ണുകള്‍ അടച്ചു. ഒരു ഗര്‍ഭമൊന്നും എന്നെക്കൊണ്ട് താങ്ങാന്‍ പറ്റില്ലെന്ന് തോന്നിയ സമയമായിരുന്നു അത്. എല്ലാ ഗര്‍ഭിണികള്‍ക്കും ഉണ്ടാകുന്നത് പോലെ തന്നെയുള്ള സംഗതികള്‍ എനിക്കും ഉണ്ടായിരുന്നു. വയ..."
3,datasets/magazine_raw/mag100.txt,magazine_text,2508,"കൊവിഡ് സൃഷ്ടിച്ച സാമ്പത്തിക പ്രതിസന്ധിയിൽ നിന്ന് കരകയറാൻ പാടുപ്പെടുകയാണ് ലോകം. മഹാമാരിയെ നിയന്ത്രിക്കാൻ വാക്‌സിൻ മാത്രമാണ് ശാശ്വത പരിഹാരമെന്ന് കണ്ട് പല രാജ്യങ്ങളും ജനങ്ങളെ കൊണ്ട് വാക്‌സിൻ എടുപ്പിക്കാനുള്ള ശ്രമത്തിലാണ്. പല സ്ഥലങ്ങളിലും ആളുകൾ വാക്‌സിൻ എടുക്കാൻ ഉത്സാഹം കാണിക്കുന്നില്ല എന്നത് സർക്കാരിനെ കൂടുതൽ പ്രശ്‍നത്തിലാക്കുന്നു. അതുകൊണ്ട് തന്നെ വാക്‌സിൻ എടുക്കാൻ തയ്യാറാകുന്നവർക്ക് പല ഓഫറുകളും ഭരണകൂടം വാഗ്ദാനം ചെയ്യുന്നു. യുഎസിലെ വെസ്റ്റ് വിർജീനിയ ഭരണകൂടം ആളുകളെ വാക്സിൻ എടുപ്പിക്കാനായി ഒരു ലോട്ടറി പദ്ധതി ആവിഷ്കരിച്ചിരിക്കയാണ്. വിജയിയ്ക്ക് പണം മാത്രമല്ല, മറിച്ച് റൈഫിളുകളും തോക്കുകളും വരെ സമ്..."


## Checking impurity

In [6]:
RE_SUSPICIOUS = re.compile(r'\(.*?\)|[a-zA-Z]')

def check_impurity(content: str, min_len = 200):
    '''returns the share of suspicious characters in a text'''
    len_text = len(content)
    if content == None or len_text < min_len:
        return 0
    return len(RE_SUSPICIOUS.findall(content)) / len_text


In [7]:
mag['impurity'] = mag['content'].progress_apply(check_impurity)

100%|██████████| 7863/7863 [00:01<00:00, 7258.47it/s]


In [8]:
mag.sort_values(by = 'impurity', ascending=False).head(5)

,file_path,dataset_name,length,content,impurity
7389,datasets/magazine_raw/mag7649.txt,magazine_text,1736,"രാജ്യം സ്വാതന്ത്ര്യ ദിനം ആഘോഷിക്കുമ്പോള്‍, നാം നിര്‍ബന്ധമായും കണ്ടിരിക്കേണ്ട കുറച്ചു ചിത്രങ്ങള്‍ ഇതാ ഇവിടെ. ഇന്ത്യയുടെ ആദ്യത്തെ സ്വാതന്ത്ര്യ ദിനാഘോഷ വേളയിലുള്ള ഈ ചിത്രങ്ങള്‍ ട്വിറ്ററിലാണ് ഷെയര്‍ ചെയ്തത്. കാണാം, ആ ചിത്രങ്ങള്‍: 15 August 1947 :: Scene on Independence Day in Delhi pic.twitter.com/Ra9Ygb5qlM 15 August 1947 :: Crowd Gathered at Delhi’s Red Fort #India जय हिन्द pic.twitter.com/dp02uHOk6B 15 August 1947 :: Scene at Red Fort on First Independence Day (Photo - Homai Vyarawalla ) #India pic.twitter.com/ZhoNrZrjHq 15 August 1947 :: Citizens in Calcutta celebrating Independence day...",0.591014
7500,datasets/magazine_raw/mag7749.txt,magazine_text,7809,കാലിഫോര്‍ണിയ: ഗ്ലാമറിന്റെ ലോകം ഉപേക്ഷിച്ച് ആത്മീയ ജീവിതത്തിലേക്ക് തിരിഞ്ഞ പ്രശസ്ത മോഡല്‍ സോഫിയ ഹയാത്ത് ഇപ്പോള്‍ ഇന്ത്യന്‍ ആത്മീയതയില്‍ മുഴുകിയിരിക്കുകയാണ്. ബിഗ് ബോസ് റിയാലിറ്റി ഷോയിലൂടെ പ്രശസ്തയായ സോഫിയാ ഹയാത്ത് ഈയിടെ നടത്തിയ ഇന്ത്യന്‍ യാത്രകളുടെ അനുഭവം ഇന്‍സ്റ്റഗ്രാമിലൂടെ പങ്കുവെച്ചു. ഔറംഗബാദിലെ കൈലാശ് ക്ഷേത്രത്തില്‍ നടത്തിയ യാത്രയിലുടനീളം ശിവഭക്തിയുടെ ലഹരിയിലായിരുന്നു താനെന്ന് അവര്‍ ഇന്‍സ്റ്റഗ്രാമില്‍ എഴുതി. ശരീര പ്രദര്‍ശനത്തിലൂടെ വാര്‍ത്തകളില്‍ നിറഞ്ഞ സോഫിയ ഹയാത്ത് സന്യാസിനി ആയി. താന്‍ സന്യാസ ജീവിതത്തിലേക്ക് മാറിയതായി സോഫിയ ഇന്‍സ്റ്റഗ്രാമിലൂടെയാണ് നേരത്തെ അറിയിച്ചത്.ഇത് വലിയ വാര്‍ത്തയാ...,0.576770
5429,datasets/magazine_raw/mag5885.txt,magazine_text,3058,"അഞ്ചുവയസുകാരിയായ ഈ നൈജീരിയന്‍ പെണ്‍കുട്ടി സകല മോഡലുകളെയും പിറകിലാക്കി സോഷ്യല്‍ മീഡിയ കീഴടക്കുകയാണ്. 'ലോകത്തിലെ തന്നെ സുന്ദരി' എന്ന് വിളിക്കാന്‍ തോന്നുന്ന ആ പെണ്‍കുട്ടിയുടെ പേര് ജാരെ എന്നാണ്. പ്രൊഫഷണല്‍ മോഡലുകളെ വെല്ലുന്ന നോട്ടവും ഇരിപ്പും അവളെ പ്രത്യേകതയുള്ളവളാക്കുന്നു. J A R E @the_j3_sisters CHILDREN ARE A GIFT FROM HEAVEN. I’m pretty excited and elated about my new style of child portraiture . Before I thought of Photographing the @the_j3_sisters , I have had frivolous ideas of how I can make artsy portraits of kids And do away from the norm . The opportunity came and I had to take a b...",0.466645
7410,datasets/magazine_raw/mag7668.txt,magazine_text,4918,"തിരുവനന്തപുരം: വ്യത്യസ്ത ലൈംഗികതയുടെ സൗന്ദര്യം വിളിച്ചോതി, മാനവീയം വീഥിയില്‍ നടന്ന ലൈംഗിക സ്വാഭിമാന ഘോഷയാത്ര വ്യത്യസ്തമായ ഒരനുഭവമായിരുന്നു. ഈ മാസം 12ന് കോഴിക്കോട് നടക്കുന്ന ഏഴാമത് ലൈംഗിക സ്വാഭിമാന ഘോഷയാത്രക്ക് മുന്നോടിയായാണ് തിരുവനന്തപുരത്ത് രണ്ടായിരത്തോളം പേര്‍ ഒത്തുചേര്‍ന്നത്. കൂട്ടായ്മക്ക് പിന്തുണ പ്രഖ്യാപിച്ച് മന്ത്രി കടകംപള്ളി സുരേന്ദ്രനും പ്രമുഖ എഴുത്തുകാരന്‍ സക്കറിയ അടക്കമുള്ള സാംസ്‌കാരിക പ്രവര്‍ത്തകരും എത്തിയിരുന്നു. 'വിമത ലൈംഗികത' എന്ന പുസ്തകം ശീതളിന് നല്‍കി സക്കറിയ പ്രകാശനം ചെയ്തു.\n \nമാനവീയം വീഥിയില്‍ നിന്നും എബി തരകന്‍ പകര്‍ത്തിയ ചിത്രങ്ങളും വീഡിയോകളുമാണ് ഇവിടെ. Kera...",0.458723
4705,datasets/magazine_raw/mag5232.txt,magazine_text,1965,"ദില്ലി: മഞ്ഞുകാലമായി, വാര്‍ഡ്രോബില്‍ നിന്നും സ്വെറ്ററടക്കമുള്ള വസ്ത്രങ്ങള്‍ പുറത്തെടുക്കാന്‍ സമയമായി. പ്രത്യേകിച്ച് ഉത്തരേന്ത്യന്‍ നഗരങ്ങളില്‍. മനുഷ്യര്‍ ഷാളും, സ്വെറ്ററും ഒക്കെയായിട്ടാണ് മഞ്ഞുകാലത്തെ മറികടക്കുന്നത്. ഈ സമയത്ത് ഏറ്റവുമധികം തണുത്ത് വലയുന്നത് തെരുവില്‍ ജീവിക്കുന്ന മൃഗങ്ങളാണ്. ഹൌസ് ഓഫ് സ്ട്രേ ആനിമല്‍സ്, തുടങ്ങിയ എന്‍.ജി.ഒകള്‍ തെരുവുനായകളെ തണുപ്പില്‍ നിന്ന് രക്ഷിക്കാനായി എത്തിയിരിക്കുകയാണ്. ഇതിനായി വൂളന്‍ പുതപ്പുകള്‍, സ്വെറ്ററുകള്‍, ബെഡ്ഡുകള്‍ എന്നിവയാണ് നല്‍കുന്നത്. തണുപ്പിനെ മറികടക്കാന്‍ ഇങ്ങനെയാണ് അവര്‍ പ്രിയപ്പെട്ട മൃഗങ്ങളെ സഹായിക്കുന്നത്. ഹൌസ് ഓഫ് സ്ട്രേ ആനിമല്‍സ് പ്രവര...",0.450382


s

In [105]:
print("----------------------------------------------------------\n")
print("Total Files: ", len(mag))
print("Total Not Pure Files: ", len(mag[mag['impurity'] >= .03]))
print("\n----------------------------------------------------------")

----------------------------------------------------------

Total Files:  7863
Total Not Pure Files:  324

----------------------------------------------------------


In [ ]:
mag[mag['impurity'] >= .03].to_csv(BASE_PATH/'magazine_files.csv', index = False)